# Feature Engineering Part 1

On this Noteboook, the process of Feature Enginnering and Feature Selection is debugged using reduced data. After the definition of the process, all the code generated is applied to the complete data on Part 2.

## Libraries

In [1]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data, transform_data

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

# Feature Selection Libraries
from cnr_methods import LOFO_GPU_Importance
import xgboost as xgb

#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data

For this pipeline, only Training Set will be used.

In [2]:
full_data, y_train = get_simplified_data()
train_data = full_data[full_data['Set']=='Train']

In [3]:
train_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [4]:
WF = 'WF1'
data = train_data[train_data['WF']==WF]
y_train = y_train[y_train.index.isin(data['ID'])]

In [5]:
data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train
...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,6235,WF1,-2.038990,-7.244520,-0.077702,-2.412487,280.165000,0.000000,Train
2019-01-15 21:00:00,6236,WF1,-1.453913,-5.145231,-0.291905,-2.148108,280.748363,0.000000,Train
2019-01-15 22:00:00,6237,WF1,-2.378300,-5.825480,-0.559868,-1.926407,279.299000,0.000000,Train


## Feature Creation

First, using the Zonal and Meridional Components of Wind, the Magnitude and Direction of Wind Vector for 100m and 10m height.

### Wind Speed Vector

In [6]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])

Changing Reference for Negative Angles:

In [7]:
feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

Using the original Features, we will create some variables over the Numerical Variables from the simplified data.

In [8]:
features = ['T', 'CLCT', 'U_100m','V_100m','U_10m','V_10m']

### Time-Relative Variables

Here,  Values for Last Week and Month for each Numerical Feature are generated.

In [9]:
for column in features:
    feature_data[column + '_last_week'] = feature_data[column].shift(7)
    feature_data[column + '_last_month'] = feature_data[column].shift(30)

Now, Month and Quarter Statistics(Mean,Median,Variance) are generated:

In [10]:
feature_data['Month_Number'] = feature_data.index.month
feature_data['Quarter_Number'] = feature_data.index.quarter

In [11]:
# Month
mean_month = feature_data.groupby('Month_Number').mean()[features]
median_month = feature_data.groupby('Month_Number').median()[features]
variance_month = feature_data.groupby('Month_Number').var()[features]

# Quarter
mean_quarter = feature_data.groupby('Quarter_Number').mean()[features]
median_quarter = feature_data.groupby('Quarter_Number').median()[features]
variance_quarter = feature_data.groupby('Quarter_Number').var()[features]

In [12]:
# Month
mean_month.columns = mean_month.columns + '_Month_Mean'
median_month.columns = median_month.columns + '_Month_Median'
variance_month.columns = variance_month.columns + '_Month_Variance'

# Quarter
mean_quarter.columns = mean_quarter.columns + '_Quarter_Mean'
median_quarter.columns = median_quarter.columns + '_Quarterh_Median'
variance_quarter.columns = variance_quarter.columns + '_Quarter_Variance'

In [13]:
# Month
feature_data = feature_data.merge(mean_month,on='Month_Number',how='left')
feature_data = feature_data.merge(median_month,on='Month_Number',how='left')
feature_data = feature_data.merge(variance_month,on='Month_Number',how='left')

# Quarter
feature_data = feature_data.merge(mean_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(median_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(variance_quarter,on='Quarter_Number',how='left')

In [14]:
feature_data.index = data.index

For periodical Features, here represented by days (Of Month, Week and Year), hour and minutes, the features are applied to sinusoidal functions to replicate the cyclic nature of the variables.

In [15]:
day = feature_data.index.day
hour = feature_data.index.hour
minute = feature_data.index.minute
dayofweek = feature_data.index.dayofweek
dayofyear = feature_data.index.dayofyear

In [16]:
days_in_month = feature_data.index.days_in_month

In [17]:
feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

feature_data["cos_hour"], feature_data["sin_hour"] = (
    np.cos(2 * np.pi * hour / 24),
    np.sin(2 * np.pi * hour / 24),
    )

feature_data["cos_minute"], feature_data["sin_minute"] = (
    np.cos(2 * np.pi * minute / 60),
    np.sin(2 * np.pi * minute / 60),
)

feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
    np.cos(2 * np.pi * (dayofyear - 1) / 365),
    np.sin(2 * np.pi * (dayofyear - 1) / 365),
)

feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
    np.cos(2 * np.pi * dayofweek / 7),
    np.sin(2 * np.pi * dayofweek / 7),
)

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [18]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

### Rolling Window Statistics

In [19]:
for column in features:
    feature_data[column + '_Rolling_7_Window_Mean'] = feature_data[column].rolling(window=7).mean()
    feature_data[column + '_Rolling_14_Window_Mean'] = feature_data[column].rolling(window=14).mean()
    feature_data[column + '_Rolling_7_Window_Variance'] = feature_data[column].rolling(window=7).var()
    feature_data[column + '_Rolling_14_Window_Variance'] = feature_data[column].rolling(window=14).var()

### Expanding Window Statistics

In [20]:
for column in features:
    feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().min()
    feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().max()

### Interaction Features

In [21]:
for base_feature in features:
    other_features = features[features.index(base_feature) + 1:]
    for alt_feature in other_features:
        feature_data[base_feature + ' x ' + alt_feature] = feature_data[base_feature] * feature_data[alt_feature]

## Feature Selection

Finally, all the Features Generated have to be filtered, so only the most relevant ones are passed to the model, as a way of avoiding Overfitting. For this process, a process called LOFO (Leave One Feature Out) is selected.

The process is simple: For each Feature, a arbitrary Model (Here a XGBoost) is cross validated on a dataset that contains all the features except one, which importance is being measured, and the precision without that feature is compared to a baseline, where all features are present. The features whose removal leads to worse results are considered most important features to the dataset.

More info about this process can be found on https://github.com/aerdem4/lofo-importance.

The implementation of this method on this however, was an adaptation of the original code to allow use of GPU resources as a faster way to obtain results, since this work involves a big number of columns.

In [24]:
final_features = feature_data.drop(features,axis=1)
final_features = final_features.rename({'key_0':'Date'},axis=1)

In [27]:
features = final_features.drop(['ID','WF','Set'],axis=1).columns

In [28]:
param = {'tree_method' : 'gpu_hist'}

In [29]:
importance_df = LOFO_GPU_Importance(final_features,y_train,features,param)

1/121 3.990001 s/it
2/121 4.775022 s/it
3/121 3.964002 s/it
4/121 3.402998 s/it
5/121 3.483516 s/it
6/121 3.618510 s/it
7/121 3.769534 s/it
8/121 3.692511 s/it
9/121 3.774998 s/it
10/121 4.164554 s/it
11/121 3.627024 s/it
12/121 4.18757 s/it
13/121 3.498022 s/it
14/121 3.570044 s/it
15/121 4.357095 s/it
16/121 4.498050 s/it
17/121 3.756070 s/it
18/121 3.734067 s/it
19/121 3.686024 s/it
20/121 3.736025 s/it
21/121 3.725510 s/it
22/121 3.723511 s/it
23/121 3.779999 s/it
24/121 3.701534 s/it
25/121 3.714707 s/it
26/121 3.690432 s/it
27/121 3.663854 s/it
28/121 3.739513 s/it
29/121 3.670023 s/it
30/121 3.728513 s/it
31/121 3.685028 s/it
32/121 3.680512 s/it
33/121 3.652036 s/it
34/121 3.660022 s/it
35/121 3.632040 s/it
36/121 3.654003 s/it
37/121 3.650024 s/it
38/121 3.605997 s/it
39/121 3.743534 s/it
40/121 3.722021 s/it
41/121 3.599021 s/it
42/121 3.669020 s/it
43/121 3.612023 s/it
44/121 3.604000 s/it
45/121 3.591507 s/it
46/121 3.860790 s/it
47/121 3.673002 s/it
48/121 3.728022 s/it
49

In [30]:
importance_df.to_csv(r'C:\Users\andre_\OneDrive\Documentos\Feature Selection\Importance_WF1.csv')

In [31]:
importance_df

,feature,score
119,V_100m x V_10m,-2.765291
3,Wind Direction 10m,-2.644290
1,Wind Direction 100m,-2.615374
117,U_100m x V_10m,-2.413561
116,U_100m x U_10m,-2.282210
...,...,...
65,T_Distance_Min,0.100482
67,CLCT_Distance_Min,0.105292
86,U_100m_Rolling_7_Window_Variance,0.118576
104,U_10m_Expanded_Window_Min,0.160317
